# Go

## Where Were We?

1. Language primitives (i.e., building blocks of languages)
2. **Language paradigms** (i.e., combinations of language primitives)
    - Last time: concurrency in TypeScript via message passing and shared memory
    - This time: **first-class concurrency** in Go
3. Building a language (i.e., designing your own language)

## Goals

1. Introduce Go as a language built for message-passing concurrency.
2. Introduce the abstraction of a **channel**.

## Go?

Go language: [https://golang.org/doc/](https://golang.org/doc/)

Go in Jupyter: [https://github.com/gopherdata/gophernotes](https://github.com/gopherdata/gophernotes)

## Go Introduction

### Recall what to look for when learning a new language

Template
1. **Input/Output** with the outside world
2. Define **data**
3. Define **code** (i.e., functions, methods, operations) on data structures

It is also beneficial to look at `Go` in comparison with `TypeScript`.

### Input/Ouput

#### Obligatory Hello World

In [1]:
import "fmt"  // fmt -> format

func main() {  // 1. func -> function in TypeScript
    fmt.Println("Hello, World!")  // 2. fmt.Println -> console.log in TypeScript
    // 3. you may be wondering why Println is capitalized?
}

main()

Hello, World!


### Compare with TypeScript

```ts
function main() {
    console.log("Hello, World!");
}

main();
```

In [2]:
// 1. Capital letter in function name indicates `export`
//    Equivalent in TypeScript is: export function hello
func Hello(name string) string {  // 2. export function hello(name: string): string in TypeScript
    // 3. let message: string = undefined; in TypeScript
    var message string  // 4. type required
    // 5. message = `Hi, ${name}. Welcome!`; in TypeScript
    message = fmt.Sprintf("Hi, %v. Welcome!", name)
    return message
}

Hello("CSC600")

Hi, CSC600. Welcome!

In [3]:
import "os"

// Write File
d1 := []byte("hello\ngo\n")  // C??
f, err := os.OpenFile("./tmp/notes.txt", os.O_RDWR|os.O_CREATE, 0755) // 0755 linux/unix permissions
f.Write(d1)
f.Close()

In [4]:
// Read File
f, err := os.OpenFile("./tmp/notes.txt", os.O_RDWR, 0755)
d2 := make([]byte, 3)  // length 3 byte array
f.Read(d2)
fmt.Print(string(d2))
f.Read(d2)
fmt.Print(string(d2))
f.Read(d2)
fmt.Print(string(d2))
f.Close()

hello
go


### Syntactic sugar for variable assignment

**Syntactic sugar** means shorthand

In [5]:
func Hello2(name string) string {
    // Equivalent code
    // var message string  // 4. type required
    // message = fmt.Sprintf("Hi, %v. Welcome!", name)
    message := fmt.Sprintf("Hi, %v. Welcome!", name)   // write it this way if you can
    return message
}

Hello("CSC600")

Hi, CSC600. Welcome!

### Data-Structures + Code on Data-Structures

#### Arrays/Slices + Loops

In [6]:
numbers := []int{ 41, 42, 43 }  // notice the { }
numbers

[41 42 43]

#### Compare with TypeScript

```ts
const numbers: int[] = [41, 42, 43];
```
or
```ts
const numbers = [41, 42, 43];
```

In [7]:
numbers[1]

42

In [8]:
numbers = append(numbers, 44)  // "pure" append
numbers

[41 42 43 44]

### Compare with TypeScript

```ts
numbers.push(44)
```
but actually
```ts
numbers = numbers.concat(44)
```

In [9]:
for i, x := range numbers {
    fmt.Println("Index", i, "Element", x)
}

Index 0 Element 41
Index 1 Element 42
Index 2 Element 43
Index 3 Element 44


#### Compare with TypeScript

```ts
numbers.forEach((i, x) => {
    console.log("Index", i, "Element", x);
});
```

#### Dictionaries

In [10]:
dictionary := make(map[string]int)
dictionary

map[]

In [11]:
dictionary["a"] = 1
dictionary

map[a:1]

In [12]:
dictionary["b"] = 2
dictionary

map[a:1 b:2]

#### Compare with TypeScript

```ts
const dictionary = {};
dictionary["a"] = 1;
dictionary["b"] = 2;
```

In [13]:
dictionary["a"]

1 true

In [14]:
dictionary["b"]

2 true

In [15]:
dictionary["c"]

0 false

In [16]:
if val, found := dictionary["c"]; found {
    fmt.Println("Found", "c", val)
} else {
    fmt.Println("Not found", "c")
}

Not found c


#### Compare with TypeScript

```ts
if ("c" in dictionary) {
    console.log("Found", "c", dictionary["c"]);
} else {
    console.log("Not found", "c");
}
```

### Algebraic-Data Types + Recursion?

In [17]:
type Leaf struct {
}

type Node struct {
    val int;
    left Tree;
    right Tree;
}

type Tree interface {
    isTree()
}
func (_ Leaf) isTree() {}
func (_ Node) isTree() {}

#### Compare with TypeScript

```ts
type Leaf = {
    tag: "LEAF"
};
type Node = {
    tag: "NODE",
    val: int,
    left: Tree,
    right: Tree,
};
type Tree = Leaf | Node;
```

In [18]:
func treeToString(tr Tree) string {
    switch nt := tr.(type) {
    case Leaf: {
        return "()"
    }
    case Node: {
        return fmt.Sprintf("(%d %s %s)", nt.val, treeToString(nt.left), treeToString(nt.right))
    }
    }
}

treeToString(Node{1, Leaf{}, Leaf{}})

(1 () ())

#### Compare with TypeScript

```ts
function treeToString(tr: Tree): string {
    switch(tr.tag) {
        case "LEAF": {
            return "()";
        }
        case "NODE": {
            return `(${tr.val} ${treeToString(tr.left)} ${treeToString(tr.right)})`;
        }
    }
}
```

### First-Class Functions?

In [19]:
func addOne(x int) int {
    return x + 1
}

In [20]:
foobar2 := addOne  // Recall the test, can we assign to a variable?
foobar2(2)

3

In [21]:
var foobar3 func(int) int  // Function type, (x: int) => int in TypeScript
foobar3 = func(x int) int {  // Anonymous functions in Go
    return x + 1
}
foobar3(2)

3

#### Compare with TypeScript

```ts
const foobar3: (y: int) => int = (x: int): int => {
    return x + 1;
};
```

## Where is Concurrency?

### Channels

In [22]:
import "time"

func worker(done chan string) {  // 1. `chan` is a Go keyword for channel
                                 // 2. `chan string` means we have a channel for sending strings
    fmt.Print("Doing work that takes one second...")
    time.Sleep(time.Second)  // "Simulate work being done for one second"
    fmt.Println("done")
    done <- "MESSAGE"  // Write "MESSAGE" to channel `done`
}

In [23]:
// Parallelism
done := make(chan string)  // 1. Create a string channel
                           // 2. We have first-class channels because we can assign channels to variables
go worker(done)            // 3. Launch the goroutine worker with channel done
time.Sleep(2*time.Second)  // 4. Hack: "worker thread" takes 1 second, so let's wait for two seconds
fmt.Println("Main thread")

Doing work that takes one second...done
Main thread


12 <nil>

In [24]:
done := make(chan string)  // 1. Create a string channel
                           // 2. We have first-class channels because we can assign channels to variables
go worker(done)            // 3. Launch the goroutine worker with channel done
<- done                    // 4. Block until the worker thread is done, i.e., synchronize so concurrency
fmt.Println("Main thread")

Doing work that takes one second...done
Main thread


12 <nil>

### Read vs. Write Channels and Parallelism

In [25]:
func fib(n int) int {  // naive implementation
    if (n == 0) {
        return 1
    } else if (n == 1) {
        return 1
    } else {
        return fib(n-1) + fib(n-2)
    }
}

In [26]:
func fibWorker(receiver <-chan int, transmitter chan<- int) {
    // 1. receiver is receive-only channel
    // 2. transmitter is send-only channel
    n := <- receiver  // 3. Read from input
    fmt.Println("Calling fibWorker...", n)
    
    // Run fibonacci
    res := fib(n)
    
    fmt.Println("Result for", n, "...", res)
    transmitter <- res  // 4. Write to output
}

### 1 Logical Worker

In [27]:
// Create channels
receiver := make(chan int)
transmitter := make(chan int)

// "Launch worker thread"
go fibWorker(receiver, transmitter)
fmt.Println("Launching logical equivalent worker thread...")
receiver <- 30

// Block until we get output
output := <- transmitter
fmt.Println("Recieved", output)

Launching logical equivalent worker thread...
Calling fibWorker... 30
Result for 30 ... 1346269
Recieved 1346269


17 <nil>

### 2 Logical Workers

In [28]:
// Create channels
receiver1 := make(chan int)
transmitter1 := make(chan int)
receiver2 := make(chan int)
transmitter2 := make(chan int)

// "Launch worker thread"
go fibWorker(receiver1, transmitter1)
go fibWorker(receiver2, transmitter2)
fmt.Println("Launching logical equivalent of worker threads...")
receiver1 <- 30
receiver2 <- 25

// Block until we get output
output1 := <- transmitter1
output2 := <- transmitter2
fmt.Println("Recieved", output1)
fmt.Println("Recieved", output2)

Launching logical equivalent of worker threads...
Calling fibWorker... 30
Calling fibWorker... 25
Result for 25 ... 121393
Result for 30 ... 1346269
Recieved 1346269
Recieved 121393


16 <nil>

### N Logical Workers for Parallelism

In [29]:
var transmitters [](chan int)
N := 10
for i := 0; i < N; i++ {
    receiver := make(chan int)
    transmitter := make(chan int)
    
    // Launch worker
    go fibWorker(receiver, transmitter)
    receiver <- 30-2*i
    
    transmitters = append(transmitters, transmitter)
}

// Block for outputs, i.e., synchronize
for i := 0; i < N; i++ {
    output := <- transmitters[i]
}

Calling fibWorker... 30
Calling fibWorker... 28
Calling fibWorker... 26
Calling fibWorker... 24
Calling fibWorker... 22
Calling fibWorker... 20
Calling fibWorker... 18
Calling fibWorker... 16
Calling fibWorker... 14
Calling fibWorker... 12
Result for 12 ... 233
Result for 14 ... 610
Result for 16 ... 1597
Result for 18 ... 4181
Result for 20 ... 10946
Result for 22 ... 28657
Result for 24 ... 75025
Result for 26 ... 196418
Result for 28 ... 514229
Result for 30 ... 1346269


### Channels, Mutexes, and Producer/Consumer

Mental model
1. One shared message box
2. **Producers** put messages in the message box
    * User clicks screen, producing a message in the message box: "click at x, y"
3. **Consumers** take message out of the message box and process them
    * Browser consumer takes message out of the messabe box, and fires "onClick" callback

In [30]:
import "sync"

type WorkQueue struct {
    sync.Mutex
    queue []string
}

var queue []string
workQueue := WorkQueue{
    queue: queue,
}

In [31]:
func produce(workQueue *WorkQueue, input chan string) {
    task := <- input
    workQueue.Lock()
    defer workQueue.Unlock()  // Call at the end of the function
    workQueue.queue = append(workQueue.queue, task)
}

In [32]:
input := make(chan string)
go produce(&workQueue, input)
input <- "hello"
workQueue.queue

[hello]

In [33]:
import "math/rand"

func taskCallback(task string) (string, time.Duration) {
    elapsed := time.Duration(rand.Intn(100))*time.Millisecond
    time.Sleep(elapsed)
    return task, elapsed
}

func consume(workQueue *WorkQueue, output chan string) {
    workQueue.Lock()
    if (len(workQueue.queue) > 0) {
        task := workQueue.queue[0]
        workQueue.queue = workQueue.queue[1:]
        workQueue.Unlock()  // Only lock for as long as we need

        // Simulate triggering callback
        res, elapsed := taskCallback(task)
        
        output <- fmt.Sprintf("%v took %d ms ...", res, elapsed.Milliseconds())
    } else {
        workQueue.Unlock()
        output <- "None"
    }
}

In [34]:
output := make(chan string)
go consume(&workQueue, output)
<- output
workQueue.queue

[]

In [35]:
N := 100
for i := 0; i < N; i++ {
    go produce(&workQueue, input)
    input <- fmt.Sprintf("task%d", i)
}

In [36]:
var outputs [](chan string)
for i := 0; i < N; i++ {
    output := make(chan string)
    go consume(&workQueue, output)
    outputs = append(outputs, output)
}

for i := 0; i < N; i++ {
    res, _ := <- outputs[i]
    fmt.Println(res)
}

task1 took 47 ms ...
task2 took 59 ms ...
task3 took 81 ms ...
task6 took 25 ms ...
task23 took 15 ms ...
task26 took 78 ms ...
task14 took 94 ms ...
task16 took 33 ms ...
task13 took 3 ms ...
task25 took 85 ms ...
task95 took 40 ms ...
task96 took 52 ms ...
task84 took 23 ms ...
task45 took 13 ms ...
task49 took 21 ms ...
task90 took 91 ms ...
task47 took 94 ms ...
task46 took 90 ms ...
task74 took 63 ms ...
task91 took 78 ms ...
task77 took 41 ms ...
task63 took 28 ms ...
task82 took 43 ms ...
task57 took 0 ms ...
task71 took 2 ms ...
task85 took 37 ms ...
task69 took 47 ms ...
task81 took 96 ms ...
task93 took 5 ms ...
task76 took 18 ms ...
task51 took 47 ms ...
task64 took 55 ms ...
task65 took 51 ms ...
task99 took 98 ms ...
task61 took 56 ms ...
task68 took 5 ms ...
task94 took 46 ms ...
task72 took 83 ms ...
task75 took 76 ms ...
task86 took 59 ms ...
task78 took 33 ms ...
task66 took 61 ms ...
task59 took 5 ms ...
task70 took 77 ms ...
task92 took 3 ms ...
task83 took 20 ms ...

## Summary

1. We saw a general purpose language called Go designed with *first-class channels*.
2. Go is primarily aimed at message-passing concurrency.
3. We also saw the producer/consumer pattern and how to use mutexes in Go.